In [31]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata, chi2
import warnings
warnings.filterwarnings('ignore')


df = pd.read_excel('dataset/0816.xlsx')
df = df.apply(pd.to_numeric, errors='coerce')  # 将非数字的值转换为 NaN
df = df.astype(float)  # 明确将所有值转换为 float 类型

# 因变量 Y1 和 Y2（前两列）
# 自变量 X1 到 Xn（其余列）
# 其中第1列和第2列是两个固定变量，其他列是交互变量
df.columns = ['Y1', 'Y2'] + [f'X{i}' for i in range(1, df.shape[1] - 1)]

# 首先，计算每一列交互变量的秩
ranked_data = df.iloc[:, 2:].apply(rankdata, axis=0)

# 将计算的秩合并回原始数据框
for col in ranked_data.columns:
    df[col] = ranked_data[col]

# 计算每一行所有交互变量的秩和
df['rank_sum'] = df.iloc[:, 2:].sum(axis=1)

# 分别计算固定变量 Y1 和 Y2 的秩和
rank_sums_Y1 = df.groupby('Y1')['rank_sum'].sum()
rank_sums_Y2 = df.groupby('Y2')['rank_sum'].sum()

# 计算交互作用的秩和
df['interaction'] = df['Y1'].astype(str) + df['Y2'].astype(str)
rank_sums_Y1Y2 = df.groupby('interaction')['rank_sum'].sum()

# 总样本量和固定变量 Y1 和 Y2 水平数量
N = len(df)
k_Y1 = len(df['Y1'].unique())
k_Y2 = len(df['Y2'].unique())

# 计算每个固定变量和交互作用的H统计量
def calc_H(rank_sums, factor_levels, N):
    H = 12 / (N * (N + 1)) * sum(
        (rank_sums[level] ** 2) / factor_levels[level]
        for level in rank_sums.index
    ) - 3 * (N + 1)
    return H

H_Y1 = calc_H(rank_sums_Y1, df['Y1'].value_counts().to_dict(), N)
H_Y2 = calc_H(rank_sums_Y2, df['Y2'].value_counts().to_dict(), N)

# 交互作用的H统计量
H_Y1Y2 = (12 / (N * (N + 1))) * sum(
    (rank_sums_Y1Y2.loc[level] ** 2) / len(df[df['interaction'] == level])
    for level in df['interaction'].unique()
) - H_Y1 - H_Y2 - 3 * (N + 1)

# 计算自由度和P值
df_Y1 = k_Y1 - 1
df_Y2 = k_Y2 - 1
df_Y1Y2 = df_Y1 * df_Y2

p_Y1 = 1 - chi2.cdf(H_Y1, df_Y1)
p_Y2 = 1 - chi2.cdf(H_Y2, df_Y2)
p_Y1Y2 = 1 - chi2.cdf(H_Y1Y2, df_Y1Y2)

# 输出结果
print(f"H_Y1 = {H_Y1:.4f}, p_Y1 = {p_Y1:.4f}")
print(f"H_Y2 = {H_Y2:.4f}, p_Y2 = {p_Y2:.4f}")
print(f"H_Y1Y2 = {H_Y1Y2:.4f}, p_Y1Y2 = {p_Y1Y2:.4f}")


H_Y1 = 291940.4626, p_Y1 = 0.0000
H_Y2 = 291449.9986, p_Y2 = 0.0000
H_Y1Y2 = -291393.3194, p_Y1Y2 = 1.0000


In [38]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata, chi2
import warnings
warnings.filterwarnings('ignore')


df = pd.read_excel('dataset/0816.xlsx')
df = df.apply(pd.to_numeric, errors='coerce')  # 将非数字的值转换为 NaN
df = df.astype(float)  # 明确将所有值转换为 float 类型

# 因变量 Y1 和 Y2（前两列）
# 自变量 X1 到 Xn（其余列）
# 其中第1列和第2列是两个固定变量，其他列是交互变量
df.columns = ['Y1', 'Y2'] + [f'X{i}' for i in range(1, df.shape[1] - 1)]

In [40]:
import pandas as pd
import numpy as np
from scipy import stats

def scheierRayHare(factor1, factor2, goal_var, df):
    """ 
    Scheier Ray Hare Test

    Input:
    factor1:    Column name in df which represents Factor 1
    factor2:    Column name in df which represents Factor 2
    goal_var:   Column name in df which represents Measurement
    df:         Dataframe

    Returns:
    A dataframe similar to R's scheirerRayHare output.
    """

    df['rank'] = df[goal_var].rank()

    # Group by rows (factor1)
    rows = df.groupby([factor1], as_index=False).agg({'rank': ['count', 'mean', 'var']}).rename(columns={'rank':'row'})
    rows.columns = ['_'.join(col) for col in rows.columns]
    rows['row_mean_rows'] = rows.row_mean.mean()
    rows['sqdev_rows'] = (rows.row_mean - rows.row_mean_rows)**2

    # Group by columns (factor2)
    cols = df.groupby([factor2], as_index=False).agg({'rank': ['count', 'mean', 'var']}).rename(columns={'rank':'col'})
    cols.columns = ['_'.join(col) for col in cols.columns]
    cols['col_mean_cols'] = cols.col_mean.mean()
    cols['sqdev_cols'] = (cols.col_mean - cols.col_mean_cols)**2

    # Group by interaction (factor1, factor2)
    data_sum = df.groupby([factor1, factor2], as_index=False).agg({'rank': ['count', 'mean', 'var']})
    data_sum.columns = ['_'.join(col) for col in data_sum.columns]

    nobs_row = rows.row_count.mean()
    nobs_total = rows.row_count.sum()
    nobs_col = cols.col_count.mean()

    Columns_SS = cols.sqdev_cols.sum() * nobs_col
    Rows_SS = rows.sqdev_rows.sum() * nobs_row
    Within_SS = data_sum.rank_var.sum() * (data_sum.rank_count.min() - 1)
    MS_total = df['rank'].var()
    TOTAL_SS = MS_total * (nobs_total - 1)
    Inter_SS = TOTAL_SS - Within_SS - Rows_SS - Columns_SS

    H_rows = Rows_SS / MS_total
    H_cols = Columns_SS / MS_total
    H_inter = Inter_SS / MS_total

    df_rows = len(rows) - 1
    df_cols = len(cols) - 1
    df_inter = df_rows * df_cols
    df_total = len(df) - 1
    df_within = df_total - df_inter - df_cols - df_rows

    p_rows = 1 - stats.chi2.cdf(H_rows, df_rows)
    p_cols = 1 - stats.chi2.cdf(H_cols, df_cols)
    p_inter = 1 - stats.chi2.cdf(H_inter, df_inter)

    # Organize results in a dataframe
    results = pd.DataFrame({
        'Df': [df_rows, df_cols, df_inter, df_within],
        'Sum Sq': [Rows_SS, Columns_SS, Inter_SS, Within_SS],
        'H': [H_rows, H_cols, H_inter, np.nan],
        'p.value': [p_rows, p_cols, p_inter, np.nan]
    }, index=[factor1, factor2, f"{factor1}:{factor2}", "Residuals"])

    return results.round(4)





In [42]:
import os


save_path = 'results'
for _X in df.filter(regex='^X').columns:
    res = scheierRayHare('Y1', 'Y2', _X, df)
    save_name = _X + '.csv'
    res.to_csv(os.path.join(save_path, save_name))